In [74]:
import pandas as pd 
import numpy as np
from sklearn import datasets, linear_model, metrics, preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import KMeans
from scipy import stats
import geopy.distance
import datetime
import csv
from predict import *
from numpy import *

In [49]:
train_data = pd.read_csv(open('data/train.csv'), nrows=100000)
print(len(train_data))
print(train_data.head())


100000
               TRIP_ID CALL_TYPE  ORIGIN_CALL  ORIGIN_STAND   TAXI_ID  \
0  1372636858620000589         C          NaN           NaN  20000589   
1  1372637303620000596         B          NaN           7.0  20000596   
2  1372636951620000320         C          NaN           NaN  20000320   
3  1372636854620000520         C          NaN           NaN  20000520   
4  1372637091620000337         C          NaN           NaN  20000337   

    TIMESTAMP DAY_TYPE  MISSING_DATA  \
0  1372636858        A         False   
1  1372637303        A         False   
2  1372636951        A         False   
3  1372636854        A         False   
4  1372637091        A         False   

                                            POLYLINE  
0  [[-8.618643,41.141412],[-8.618499,41.141376],[...  
1  [[-8.639847,41.159826],[-8.640351,41.159871],[...  
2  [[-8.612964,41.140359],[-8.613378,41.14035],[-...  
3  [[-8.574678,41.151951],[-8.574705,41.151942],[...  
4  [[-8.645994,41.18049],[-8.645949,41

In [50]:
#Data Cleaning
test_data = pd.read_csv(open('data/test_public.csv'))
print(test_data.head())



  TRIP_ID CALL_TYPE  ORIGIN_CALL  ORIGIN_STAND   TAXI_ID   TIMESTAMP DAY_TYPE  \
0      T1         B          NaN          15.0  20000542  1408039037        A   
1      T2         B          NaN          57.0  20000108  1408038611        A   
2      T3         B          NaN          15.0  20000370  1408038568        A   
3      T4         B          NaN          53.0  20000492  1408039090        A   
4      T5         B          NaN          18.0  20000621  1408039177        A   

   MISSING_DATA  
0         False  
1         False  
2         False  
3         False  
4         False  


In [51]:
train_data_clean = train_data.copy()

train_data_clean.drop("TRIP_ID", axis=1, inplace=True)
train_data_clean.drop("ORIGIN_CALL", axis=1, inplace=True)
train_data_clean.drop("ORIGIN_STAND", axis=1, inplace=True)
train_data_clean.drop("TAXI_ID", axis=1, inplace=True)
train_data_clean.drop("DAY_TYPE", axis=1, inplace=True)
train_data_clean.drop("MISSING_DATA", axis=1, inplace=True)
train_data_clean.drop("POLYLINE", axis=1, inplace=True) 


for i, row in train_data.iterrows():
    time = time_from_polyline(list_from_polyline_string(row["POLYLINE"]))
    train_data.set_value(i, "TRAVEL_TIME", time)
    
    
for i, row in train_data_clean.iterrows():
    if row["CALL_TYPE"] == "A":
        train_data_clean.set_value(i, "CALL_TYPE", 1)
    elif row["CALL_TYPE"] == "B":
        train_data_clean.set_value(i, "CALL_TYPE", 2)
    elif row["CALL_TYPE"] == "C":
        train_data_clean.set_value(i, "CALL_TYPE", 3)

print(train_data.head())
print(train_data_clean.head())

C:\Users\Pallav Gupta\AppData\Local\Programs\Python\Python36-32\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


C:\Users\Pallav Gupta\AppData\Local\Programs\Python\Python36-32\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\Pallav Gupta\AppData\Local\Programs\Python\Python36-32\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\Pallav Gupta\AppData\Local\Programs\Python\Python36-32\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


               TRIP_ID CALL_TYPE  ORIGIN_CALL  ORIGIN_STAND   TAXI_ID  \
0  1372636858620000589         C          NaN           NaN  20000589   
1  1372637303620000596         B          NaN           7.0  20000596   
2  1372636951620000320         C          NaN           NaN  20000320   
3  1372636854620000520         C          NaN           NaN  20000520   
4  1372637091620000337         C          NaN           NaN  20000337   

    TIMESTAMP DAY_TYPE  MISSING_DATA  \
0  1372636858        A         False   
1  1372637303        A         False   
2  1372636951        A         False   
3  1372636854        A         False   
4  1372637091        A         False   

                                            POLYLINE  TRAVEL_TIME  
0  [[-8.618643,41.141412],[-8.618499,41.141376],[...        330.0  
1  [[-8.639847,41.159826],[-8.640351,41.159871],[...        270.0  
2  [[-8.612964,41.140359],[-8.613378,41.14035],[-...        960.0  
3  [[-8.574678,41.151951],[-8.574705,41.151942],

  CALL_TYPE   TIMESTAMP
0         2  1408039037
1         2  1408038611
2         2  1408038568
3         2  1408039090
4         2  1408039177


C:\Users\Pallav Gupta\AppData\Local\Programs\Python\Python36-32\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  del sys.path[0]
C:\Users\Pallav Gupta\AppData\Local\Programs\Python\Python36-32\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Pallav Gupta\AppData\Local\Programs\Python\Python36-32\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  from ipykernel import kernelapp as app


In [52]:
test_data_clean = test_data.copy()

test_data_clean.drop("TRIP_ID", axis=1, inplace=True)
test_data_clean.drop("ORIGIN_CALL", axis=1, inplace=True)
test_data_clean.drop("ORIGIN_STAND", axis=1, inplace=True)
test_data_clean.drop("TAXI_ID", axis=1, inplace=True)
test_data_clean.drop("DAY_TYPE", axis=1, inplace=True)
test_data_clean.drop("MISSING_DATA", axis=1, inplace=True)
for i, row in test_data_clean.iterrows():
    if row['CALL_TYPE'] == "A":
        test_data_clean.set_value(i, "CALL_TYPE", 1)
    elif row['CALL_TYPE'] == "B":
        test_data_clean.set_value(i, "CALL_TYPE", 2)
    elif row['CALL_TYPE'] == "C":
        test_data_clean.set_value(i, "CALL_TYPE", 3)

print(test_data_clean.head())


  CALL_TYPE   TIMESTAMP
0         2  1408039037
1         2  1408038611
2         2  1408038568
3         2  1408039090
4         2  1408039177


C:\Users\Pallav Gupta\AppData\Local\Programs\Python\Python36-32\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  del sys.path[0]
C:\Users\Pallav Gupta\AppData\Local\Programs\Python\Python36-32\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Pallav Gupta\AppData\Local\Programs\Python\Python36-32\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  from ipykernel import kernelapp as app


In [54]:

train_scaled = preprocessing.scale(train_data_clean)

C:\Users\Pallav Gupta\AppData\Local\Programs\Python\Python36-32\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by the scale function.
  
C:\Users\Pallav Gupta\AppData\Local\Programs\Python\Python36-32\lib\site-packages\sklearn\preprocessing\data.py:176: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


2.7284841053187848e-17

In [57]:
test_scaled = preprocessing.scale(test_data_clean)
test_scaled[0:10]

C:\Users\Pallav Gupta\AppData\Local\Programs\Python\Python36-32\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64, object were all converted to float64 by the scale function.
  """Entry point for launching an IPython kernel.
C:\Users\Pallav Gupta\AppData\Local\Programs\Python\Python36-32\lib\site-packages\sklearn\preprocessing\data.py:176: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


array([[-0.21595628, -1.40965546],
       [-0.21595628, -1.40979113],
       [-0.21595628, -1.40980483],
       [-0.21595628, -1.40963858],
       [-0.21595628, -1.40961088],
       [-1.51984324, -1.4102577 ],
       [-0.21595628, -1.40971629],
       [-1.51984324, -1.40968381],
       [-0.21595628, -1.40980642],
       [-0.21595628, -1.40997903]])

In [60]:
kmeans = KMeans(n_clusters=15, random_state=0).fit(train_scaled)
labels = kmeans.labels_
train_data_clean['CLUSTERS'] = labels
print(train_data_clean[0:10])

  CALL_TYPE   TIMESTAMP  CLUSTERS
0         3  1372636858         3
1         2  1372637303        12
2         3  1372636951         3
3         3  1372636854         3
4         3  1372637091         3
5         3  1372636965         3
6         3  1372637210         3
7         3  1372637299         3
8         3  1372637274         3
9         3  1372637905         3


In [95]:
labels = kmeans.predict(test_scaled)

test_data_clean['CLUSTERS'] = labels
np.savetxt("clusters.csv", labels, delimiter=",")
print(test_data_clean)

    CALL_TYPE   TIMESTAMP  CLUSTERS
0           2  1408039037        12
1           2  1408038611        12
2           2  1408038568        12
3           2  1408039090        12
4           2  1408039177        12
5           1  1408037146         2
6           2  1408038846        12
7           1  1408038948         2
8           2  1408038563        12
9           2  1408038021        12
10          2  1408038267        12
11          3  1408038946         3
12          3  1408039130         3
13          3  1408036255         3
14          3  1408038388         3
15          3  1408037740         3
16          3  1408038804         3
17          3  1408038215         3
18          2  1408038749        12
19          3  1408036754         3
20          2  1408039125        12
21          1  1408039009         2
22          1  1408038924         2
23          2  1408039145        12
24          2  1408039057        12
25          2  1408039029        12
26          2  1408038859   

In [66]:
yi_train = train_data["TRAVEL_TIME"]
print(yi_train[0:10])

0    330.0
1    270.0
2    960.0
3    630.0
4    420.0
5    375.0
6    525.0
7    495.0
8    555.0
9    270.0
Name: TRAVEL_TIME, dtype: float64


In [97]:
train_data_clean["TRAVEL_TIME"] = yi_train
# print(train_data_clean[0:10])

for i in range(15):
    print("CLuster no: ", i)
    data_req = train_data_clean[train_data_clean["CLUSTERS"] == i]
    yi = data_req['TRAVEL_TIME']
    data_req.drop("TRAVEL_TIME", axis = 1, inplace=True)
    data_req.drop("CLUSTERS", axis = 1, inplace=True)
    
    # print(data_req[0:10])
    data_req_test = test_data_clean[test_data_clean["CLUSTERS"] == i]
    data_req_test.drop("CLUSTERS", axis = 1, inplace=True)
    if data_req_test.shape[0] == 0:
        continue
    
    random_forest_reg = RandomForestRegressor()
    random_forest_reg.fit(data_req, yi)
    
    predicted = random_forest_reg.predict(data_req_test)
    
    data_req_test["TRAVEL_TIME"] = predicted
    
    print(data_req_test)
    
    

CLuster no:  0


C:\Users\Pallav Gupta\AppData\Local\Programs\Python\Python36-32\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Users\Pallav Gupta\AppData\Local\Programs\Python\Python36-32\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Pallav Gupta\AppData\Local\Programs\Python\Python36-32\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


    CALL_TYPE   TIMESTAMP  TRAVEL_TIME
291         3  1419166062        294.0
293         3  1419171112        294.0
298         3  1419171985        294.0
299         3  1419171073        294.0
300         3  1419162114        294.0
301         3  1419167924        294.0
303         3  1419161986        294.0
308         3  1419171893        294.0
313         3  1419168199        294.0
314         3  1419171201        294.0
317         3  1419172121        294.0
CLuster no:  1
CLuster no:  2


C:\Users\Pallav Gupta\AppData\Local\Programs\Python\Python36-32\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Users\Pallav Gupta\AppData\Local\Programs\Python\Python36-32\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Pallav Gupta\AppData\Local\Programs\Python\Python36-32\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Pa

   CALL_TYPE   TIMESTAMP  TRAVEL_TIME
5          1  1408037146       1210.0
7          1  1408038948       1210.0
21         1  1408039009       1210.0
22         1  1408038924       1210.0
36         1  1408038095       1210.0
59         1  1408038636       1210.0
63         1  1408038842       1210.0
67         1  1408037585       1210.0
CLuster no:  3


...

    CALL_TYPE   TIMESTAMP  TRAVEL_TIME
80          1  1412065294       211.25
82          1  1412065076       211.25
83          1  1412064356       211.25
85          1  1412065159       211.25
89          1  1412065051       211.25
90          1  1412064659       211.25
93          1  1412065352       211.25
96          1  1412064343       211.25
97          1  1412064778       211.25
98          1  1412065446       211.25
105         1  1412065567       211.25
109         1  1412065352       211.25
110         1  1412065329       211.25
111         1  1412065271       211.25
113         1  1412065389       211.25
114         1  1412065379       211.25
120         1  1412065266       211.25
121         1  1412065621       211.25
122         1  1412065784       211.25
123         1  1412065476       211.25
127         1  1412065285       211.25
138         1  1412065220       211.25
139         1  1412065112       211.25
142         1  1412064430       211.25
144         1  1412065395

C:\Users\Pallav Gupta\AppData\Local\Programs\Python\Python36-32\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Pallav Gupta\AppData\Local\Programs\Python\Python36-32\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Users\Pallav Gupta\AppData\Local\Programs\Python\Python36-32\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


   CALL_TYPE   TIMESTAMP  TRAVEL_TIME
0          2  1408039037        267.6
1          2  1408038611        267.6
2          2  1408038568        267.6
3          2  1408039090        267.6
4          2  1408039177        267.6
6          2  1408038846        267.6
8          2  1408038563        267.6
9          2  1408038021        267.6
10         2  1408038267        267.6
18         2  1408038749        267.6
20         2  1408039125        267.6
23         2  1408039145        267.6
24         2  1408039057        267.6
25         2  1408039029        267.6
26         2  1408038859        267.6
27         2  1408039165        267.6
28         2  1408038439        267.6
29         2  1408038605        267.6
30         2  1408038529        267.6
31         2  1408038808        267.6
37         2  1408038796        267.6
38         2  1408039102        267.6
39         2  1408038865        267.6
40         2  1408038486        267.6
43         2  1408038310        267.6
44         2

    CALL_TYPE   TIMESTAMP  TRAVEL_TIME
155         2  1412617225        684.0
156         2  1412617178        684.0
158         2  1412616796        684.0
159         2  1412617121        684.0
163         2  1412616790        684.0
165         2  1412616610        684.0
169         2  1412617383        684.0
170         2  1412616424        684.0
171         2  1412616445        684.0
173         2  1412615860        684.0
174         2  1412617473        684.0
176         2  1412616274        684.0
180         2  1412616656        684.0
181         2  1412617407        684.0
184         2  1412616996        684.0
187         2  1412617425        684.0
188         2  1412617312        684.0
191         2  1412616826        684.0
194         2  1412616812        684.0
197         2  1412617001        684.0
199         2  1412616304        684.0
202         2  1412617227        684.0
205         2  1412616988        684.0
206         2  1412616861        684.0
210         2  1412616212

C:\Users\Pallav Gupta\AppData\Local\Programs\Python\Python36-32\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
